In [5]:
%load_ext autoreload
%autoreload 2

import os
import re
import sys

sys.path.append(os.path.abspath("../"))


import faiss
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter

pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 1000)
pd.set_option("display.max_rows", 400)


load_dotenv()


from src.data_utils import PodcastContainer, load_clean_scores
from src.player_utils import PlayerUtil
from src.utils import get_repo_root

cont = PodcastContainer()

podcast_df = cont.get_all_episodes()
display(podcast_df.head(10))

scores = load_clean_scores(["2023-24"])
scores.head(5)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


,publication_date,file_name,file_path,content,duration,podcast_name
0,2023-10-18,safest_players_to_draft_favorite_lateround_tar...,G:\My Drive\Columbia\Practical Deep Learning\F...,Welcome into a Wednesday edition of the Fantas...,3911,rotowire
1,2023-10-18,how_to_navigate_a_fantasy_basketball_draft_the...,G:\My Drive\Columbia\Practical Deep Learning\F...,Welcome to the Fantasy MBA Today podcast. To t...,0:50:53,nbatoday
2,2023-10-18,sleepers_mock_draft_review_with_alex_rikleen,G:\My Drive\Columbia\Practical Deep Learning\F...,Welcome to the Fantasy NBA Today podcast. Yes....,0:44:09,nbatoday
3,2023-10-19,players_to_avoid_who_to_take_after_jokic_favor...,G:\My Drive\Columbia\Practical Deep Learning\F...,"Hey, everybody. I'm Doc. He's Rick. We're the ...",3203,rotowire
4,2023-10-19,mock_draft_review_pick_2_must_draft_targets_wi...,G:\My Drive\Columbia\Practical Deep Learning\F...,"Welcome to the Fantasy MBA Today podcast. Man,...",0:54:54,nbatoday
5,2023-10-19,building_the_most_fun_team,G:\My Drive\Columbia\Practical Deep Learning\F...,Welcome to the Fantasy MBA Today podcast. That...,0:31:54,nbatoday
6,2023-10-19,mock_draft_review_pick_1_must_draft_targets_wi...,G:\My Drive\Columbia\Practical Deep Learning\F...,Welcome to the Fantasy MBA Today podcast. My f...,0:43:13,nbatoday
7,2023-10-20,nba_preseason_risers_fallers_202324,G:\My Drive\Columbia\Practical Deep Learning\F...,Welcome to the award winning fantasy basketbal...,1840,rotowire
8,2023-10-20,must_draft_players_for_fantasy_basketball_the_...,G:\My Drive\Columbia\Practical Deep Learning\F...,Welcome to the Fantasy MBA Today podcast. That...,1:02:59,nbatoday
9,2023-10-23,western_conference_win_total_overunder_picks_w...,G:\My Drive\Columbia\Practical Deep Learning\F...,Welcome to the rotor wire fantasy basketball p...,4359,rotowire


,season_year,game_date,gameId,matchup,teamName,teamSlug,personId,personName,minutes,fieldGoalsMade,fieldGoalsAttempted,fieldGoalsPercentage,threePointersMade,threePointersAttempted,threePointersPercentage,freeThrowsMade,freeThrowsAttempted,freeThrowsPercentage,reboundsOffensive,reboundsDefensive,reboundsTotal,assists,steals,blocks,turnovers,foulsPersonal,points,plusMinusPoints,fantasyPoints,projectedFantasyPoints,outperformed
392093,2023-24,2023-10-24,22300061,LAL @ DEN,lakers,lakers,2544,lebron james,29.016667,10,16,0.625,1,4,0.25,0,1,0.0,1,7,8,5,1,0,0,1,21,7,40.50,49.35,False
392094,2023-24,2023-10-24,22300061,DEN vs. LAL,nuggets,nuggets,201599,deandre jordan,0.000000,0,0,0.000,0,0,0.00,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0.00,11.60,False
392095,2023-24,2023-10-24,22300061,DEN vs. LAL,nuggets,nuggets,202704,reggie jackson,24.066667,3,8,0.375,2,5,0.40,0,0,0.0,0,3,3,1,1,0,2,0,8,11,14.25,19.10,False
392096,2023-24,2023-10-24,22300061,LAL @ DEN,lakers,lakers,203076,anthony davis,34.150000,6,17,0.353,1,2,0.50,4,4,1.0,1,7,8,4,0,2,2,3,17,-17,36.00,51.80,False
392097,2023-24,2023-10-24,22300061,DEN vs. LAL,nuggets,nuggets,203200,justin holiday,0.000000,0,0,0.000,0,0,0.00,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0.00,8.20,False
